# **Electricity and gas data exploration**

## 0 - Set up

###  0.1 - Libraries

In [ ]:
#packages for EDA
import numpy as np
import pandas as pd

#packages for visualisation
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

#package for import data in json format
import json

#miscellania
import warnings
warnings.filterwarnings('ignore')
sns.set() # set seaborn as default style

### 0.2 - User-defined functions 

In [ ]:
def compute_lagged_corr(df, var, lagged_var, n_lags):
    return [df[var].corr(df[lagged_var].shift(j)) for j in range(n_lags + 1)]

In [ ]:
def plot_full_corr(df, is_business, is_consumption, var, lagged_var, n_lags):
    mask_busi = df['is_business'] == is_business
    mask_cons = df['is_consumption'] == is_consumption
    mask_full = mask_busi&mask_cons
    corr_list = compute_lagged_corr(df[mask_full], 
                                    var, 
                                    lagged_var, 
                                    n_lags
                                   )

    
    plt.figure(figsize=(12,5),dpi=100)
    aux_series = pd.Series(data = corr_list, 
                           index = [lag for lag in range(n_lags + 1)]
                          )
    aux_series.plot()
    plt.title(f'Target-price Pearson correlation for is_business = {is_business} and is_consumption = {is_consumption}')
    plt.xlabel('hourly lag')
    plt.ylabel('correlation')
    plt.show()
    
    print(f"Min correlation datetime: {aux_series.min()}, {aux_series.idxmin()}")
    print(f"Max correlation datetime: {aux_series.max()}, {aux_series.idxmax()}")

In [ ]:
def make_correlation_df(df, target_var, lagged_vars, n_lags):
    """
    df: dataframe
    target_var: target variable, string
    lagged_vars: list of lagged variables, list of string
    n_lags: number of lagged periods, integer
    """

    pred_ids = pd.unique(df['prediction_unit_id'])

    n_lagged_vars = len(lagged_vars)
    corr_list = [[] for i in range(n_lagged_vars)]
    lag_list = []
    is_cons_list = []
    pred_id_list = []
    
    data_dict = {}
    
    corr_vars = [target_var] + lagged_vars

    for pred_id in pred_ids:
        for is_cons in [0, 1]:
            mask_pred_id = df.prediction_unit_id == pred_id
            mask_is_cons = df.is_consumption == is_cons
            mask_full = mask_pred_id&mask_is_cons
            aux_df = df[mask_full][corr_vars].copy()

            for i in range(n_lagged_vars):
                corr_list[i] += compute_lagged_corr(aux_df, 
                                             target_var, 
                                             lagged_vars[i], 
                                             n_lags
                                            )
            
            lag_list += [lag for lag in range(n_lags + 1)]
            is_cons_list += [is_cons]*(n_lags + 1)
            pred_id_list += [pred_id]*(n_lags + 1)

            
    for i, var in enumerate(lagged_vars):
        col_name = 'corr_' + var
        data_dict.update({col_name : corr_list[i]})
        
    data_dict.update({'hours_lag' : lag_list,
                      'is_consumption' : is_cons_list,
                      'prediction_unit_id' : pred_id_list
                     }
                    )
    
    return pd.DataFrame(data = data_dict)

## 1 - Load the data

In [ ]:
#load the data in the csv format 
elect_df = pd.read_csv('../data/electricity_prices.csv')
gas_df = pd.read_csv('../data/gas_prices.csv')

## 2 - Data exploration for electricity prices

### 2.1 - Data documentation

- `origin_date`
- `forecast_date` - Represents the start of the 1-hour period when the price is valid
- `euros_per_mwh` - The price of electricity on the day ahead markets in euros per megawatt hour.
- `data_block_id`

In [ ]:
elect_df.head()

In [ ]:
elect_df['origin_date'] = pd.to_datetime(elect_df['origin_date'])
elect_df['forecast_date'] = pd.to_datetime(elect_df['forecast_date'])

### 2.2 - Checking for missing values and duplicates

In [ ]:
elect_df.info(show_counts = True)

In [ ]:
elect_df.isnull().sum()

There are no missing values in the dataset.

In [ ]:
elect_df.duplicated().sum()

Also, no duplicated rows.

#### 2.3 - Check for Daylight Saving Time

The start and end timestamps for DST for Estonia in the years 2021, 2022 and 2023 are

- `2021-03-28 03:00:00` to `2021-10-31 04:00:00`
- `2022-03-27 03:00:00` to `2022-10-30 04:00:00`
- `2023-03-26 03:00:00` to `2023-10-29 04:00:00`

#### End DST 2021

In [ ]:
end_dst_2021 = elect_df['origin_date'].dt.strftime('%Y-%m-%d') == '2021-10-30'
elect_df[end_dst_2021][0:6]

In [ ]:
end_dst_2021 = elect_df['origin_date'].dt.strftime('%Y-%m-%d') == '2021-10-31'
elect_df[end_dst_2021][0:6]

#### Start DST 2022

In [ ]:
start_dst_2022 = elect_df['origin_date'].dt.strftime('%Y-%m-%d') == '2022-03-26'
elect_df[start_dst_2022][0:6]

In [ ]:
start_dst_2022 = elect_df['origin_date'].dt.strftime('%Y-%m-%d') == '2022-03-27'
elect_df[start_dst_2022][0:6]

#### End DST 2022

In [ ]:
end_dst_2022 = elect_df['origin_date'].dt.strftime('%Y-%m-%d') == '2022-10-29'
elect_df[end_dst_2022][0:6]

In [ ]:
end_dst_2022 = elect_df['origin_date'].dt.strftime('%Y-%m-%d') == '2022-10-30'
elect_df[end_dst_2022][0:6]

#### Start DST 2023

In [ ]:
start_dst_2023 = elect_df['origin_date'].dt.strftime('%Y-%m-%d') == '2023-03-25'
elect_df[start_dst_2023][0:6]

In [ ]:
start_dst_2023 = elect_df['origin_date'].dt.strftime('%Y-%m-%d') == '2023-03-26'
elect_df[start_dst_2023][0:6]

Hence, we see that there is no change at the start of DST, but the timestamp `02:00:00` is skipped at the end of the DST.

### 2.4 - Getting the Tallinn time zone right

We first create a dataset with the timestamps in the Tallinn time zone.

In [ ]:
dt = pd.date_range(start = '2021-09-01 00:00:00', end = '2023-06-01 02:00:00', 
              freq = 'h', tz = 'Europe/Tallinn')

datetime_df = pd.DataFrame(data={'datetime_new' : dt.tz_localize(None)})

From the fact that the timestamp `02:00:00` is skipped at the end of the DST, it seems that the electricity prices need to be shifted by one hour.

In [ ]:
elect_df['origin_date'] = elect_df['origin_date'] + pd.Timedelta(1, "h")
elect_df['forecast_date'] = elect_df['forecast_date'] + pd.Timedelta(1, "h")

We then merge the datetime dataset created above with our electricity prices dataset. So now the timestamp `03:00:00` is duplicated and at the end of the DST, but it is skipped at the start of the DST.

In [ ]:
elect_df = elect_df.merge(datetime_df, how = 'left', left_on = 'forecast_date', right_on = 'datetime_new')

In [ ]:
elect_df[(elect_df['forecast_date'].dt.strftime('%Y-%m-%d') == '2021-10-31')][0:6]

In [ ]:
elect_df[(elect_df['forecast_date'].dt.strftime('%Y-%m-%d') == '2022-03-27')][0:6]

In [ ]:
elect_df.drop(columns = 'datetime_new', inplace = True)

The final result is a dataset that is in the Tallinn time zone.

### 2.5 - Distribution

In [ ]:
plt.figure(figsize=(12,5),dpi=100)
sns.lineplot(elect_df, x = 'forecast_date', y = 'euros_per_mwh')
plt.title('Electricity prices')
plt.xlabel('forecast date')
plt.ylabel('price per MW-h')
plt.show()

In [ ]:
plt.figure(figsize=(10,6),dpi=100)
sns.boxplot(elect_df['euros_per_mwh'])
plt.title('Electricity prices boxplot')
plt.xlabel('Electricity prices')
plt.ylabel('price per MW-h')
plt.show()

In [ ]:
plt.figure(figsize=(12,5),dpi=100)
sns.distplot(elect_df['euros_per_mwh'])
plt.title('Electricity prices')
#plt.xlabel('forecast date')
#plt.ylabel('price per MW-h')
plt.show()

Hence, there is one outlier and the distribution seems to be skewed towards small values.

### 2.6 - Correlation with target

In [ ]:
target_df = pd.read_csv('../data/train_clean.csv')

In [ ]:
target_df['datetime'] = pd.to_datetime(target_df['datetime'])

In [ ]:
target_df.head()

In [ ]:
target_df = target_df.merge(datetime_df, how = 'left', left_on = 'datetime', right_on = 'datetime_new')

In [ ]:
target_df[(target_df['is_consumption']==0)&(target_df['prediction_unit_id']==0)&(target_df['datetime'].dt.strftime('%Y-%m-%d') == '2021-10-31')][0:6]

In [ ]:
target_df.drop(columns = 'datetime_new', inplace = True)

In [ ]:
elect_df['diff_euros_per_mwh'] = elect_df['euros_per_mwh'].diff(1)

In [ ]:
target_price_df = target_df.merge(elect_df, how = 'left', left_on = 'datetime', right_on = 'forecast_date')
target_price_df.dropna(inplace = True)

In [ ]:
target_price_df[(target_price_df['is_consumption']==0)&(target_price_df['prediction_unit_id']==0)&(target_price_df['datetime'].dt.strftime('%Y-%m-%d') == '2021-10-31')][0:6]

In [ ]:
hourly_lag = 24*7*4*3

#### Plots of the target-electricity price Pearson correlation vs. hourly lag 

In [ ]:
plot_full_corr(target_price_df, 0, 0, 'target', 'euros_per_mwh', hourly_lag)

In [ ]:
plot_full_corr(target_price_df, 0, 1, 'target', 'euros_per_mwh', hourly_lag)

In [ ]:
plot_full_corr(target_price_df, 1, 0, 'target', 'euros_per_mwh', hourly_lag)

In [ ]:
plot_full_corr(target_price_df, 1, 1, 'target', 'euros_per_mwh', hourly_lag)

#### Plots of the target-diff electricity price Pearson correlation vs. hourly lag 

In [ ]:
plot_full_corr(target_price_df, 0, 0, 'target', 'diff_euros_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_price_df, 0, 1, 'target', 'diff_euros_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_price_df, 1, 0, 'target', 'diff_euros_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_price_df, 1, 1, 'target', 'diff_euros_per_mwh', 24*7*4*6)

It seems that the target is more strongly correlated with actual prices rather than with their difference.

#### Create a target-electricity price correlation dataframe for each target time series 

We now compute the Pearson correlation between consumption/production and lagged electricity prices as a function of the hourly lag for individual prediction unit ids time series.

In [ ]:
corr_df = make_correlation_df(target_price_df, 'target', ['euros_per_mwh'], 24*7*4*6)

In [ ]:
corr_df.head()

In [ ]:
corr_df.to_csv('../data/corr_target_elect_prices.csv', index = False)

#### Plot of the Pearson correlation between production and lagged electricity prices vs. hourly lag

In [ ]:
mask = corr_df['is_consumption'] == 0
g = sns.FacetGrid(corr_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="correlation")
g.add_legend()
g.fig.suptitle('Pearson correlation between production and lagged electricity prices')
g.fig.subplots_adjust(top=0.975)

In [ ]:
mask = (corr_df['is_consumption'] == 0)&(corr_df['hours_lag'] <= 24*7)
g = sns.FacetGrid(corr_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="correlation")
g.add_legend()
g.fig.suptitle('Pearson correlation between production and lagged electricity prices')
g.fig.subplots_adjust(top=0.975)

#### Plot of the Pearson correlation between consumption and lagged electricity prices vs. hourly lag

In [ ]:
mask = corr_df['is_consumption'] == 1
g = sns.FacetGrid(corr_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="correlation")
g.add_legend()
g.fig.suptitle('Pearson correlation between consumption and lagged electricity prices')
g.fig.subplots_adjust(top=0.975)

In [ ]:
mask = (corr_df['is_consumption'] == 1)&(corr_df['hours_lag'] <= 24*7)
g = sns.FacetGrid(corr_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="correlation")
g.add_legend()
g.fig.suptitle('Pearson correlation between consumption and lagged electricity prices')
g.fig.subplots_adjust(top=0.975)

The target-electricity prices correlation shows quite a lot of structure with respect to the lagged time. There is a clear hourly seasonality that might be related to the natural hourly seasonality of energy consumption and solar energy production. For many of the `prediction_unit_id`'s there is also a monthly seasonality, which might be related to the seasons of the year or so quartley planning in the offer and demand.  

## 3 - Data exploration for gas prices

### 3.1 - Data documentation


 - `origin_date` - The date when the day-ahead prices became available.
 - `forecast_date` - The date when the forecast prices should be relevant.
 - `[lowest/highest]_price_per_mwh` - The lowest/highest price of natural gas that on the day ahead market that trading day, in Euros per megawatt hour equivalent.
 - `data_block_id`

In [ ]:
gas_df.head()

In [ ]:
gas_df['origin_date'] = pd.to_datetime(gas_df['origin_date'])
gas_df['forecast_date'] = pd.to_datetime(gas_df['forecast_date'])

### 3.2 - Checking for missing values and duplicates

In [ ]:
gas_df.info(show_counts = True)

In [ ]:
gas_df.isnull().sum()

There are no missing values in the dataset.

In [ ]:
gas_df.duplicated().sum()

Also, no duplicated rows.

### 3.3 - Distribution

In [ ]:
plt.figure(figsize=(12,5),dpi=100)
sns.lineplot(gas_df, x = 'forecast_date', y = 'lowest_price_per_mwh', label = 'lowest price')
sns.lineplot(gas_df, x = 'forecast_date', y = 'highest_price_per_mwh', label = 'highest price')
plt.title('Gas prices time series')
plt.xlabel('forecast date')
plt.ylabel('price per MW-h')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,6),dpi=100)
sns.boxplot(gas_df[['lowest_price_per_mwh', 'highest_price_per_mwh']])
plt.title('Gas prices boxplot')
plt.xlabel('Gas prices')
plt.ylabel('price per MW-h')
plt.show()

In [ ]:
plt.figure(figsize=(12,5),dpi=100)
sns.distplot(gas_df['lowest_price_per_mwh'], label = 'lowest price')
sns.distplot(gas_df['highest_price_per_mwh'], label = 'highest price')
plt.title('Gas prices ditribution')
plt.xlabel('gas price per MW-h')
#plt.ylabel('price per MW-h')
plt.legend()
plt.show()

Hence, there is no outlier and the distributions seems to be skewed towards small values.

### 3.4 - Correlation with target

In [ ]:
gas_df['mean_price_per_mwh'] = 0.5*(gas_df['lowest_price_per_mwh'] + gas_df['highest_price_per_mwh'])
gas_df['var_price_per_mwh'] = gas_df['highest_price_per_mwh'] - gas_df['lowest_price_per_mwh']

In [ ]:
target_df['date'] = pd.to_datetime(target_df['datetime'].dt.date)
target_df.info()

In [ ]:
target_gas_price_df = target_df.merge(gas_df, how = 'left', left_on = 'date', right_on = 'forecast_date')
target_gas_price_df.dropna(inplace = True)

In [ ]:
target_gas_price_df[(target_gas_price_df['is_consumption']==0)&(target_gas_price_df['prediction_unit_id']==0)&(target_gas_price_df['datetime'].dt.strftime('%Y-%m-%d') == '2021-10-31')][0:6]

#### Plots of the target-gas price Pearson correlation vs. hourly lag 

##### Correlation with mean price

In [ ]:
plot_full_corr(target_gas_price_df, 0, 0, 'target', 'mean_price_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_gas_price_df, 0, 1, 'target', 'mean_price_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_gas_price_df, 1, 0, 'target', 'mean_price_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_gas_price_df, 1, 1, 'target', 'mean_price_per_mwh', 24*7*4*6)

##### Correlation with highest price

In [ ]:
plot_full_corr(target_gas_price_df, 0, 0, 'target', 'highest_price_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_gas_price_df, 0, 1, 'target', 'highest_price_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_gas_price_df, 1, 0, 'target', 'highest_price_per_mwh', 24*7*4*6)

In [ ]:
plot_full_corr(target_gas_price_df, 1, 1, 'target', 'highest_price_per_mwh', 24*7*4*6)

Overall, it seems that the the target-gas price correlation does not vary mush with respect to the hourly lag.

#### Create a target-gas price correlation dataframe for each target time series 

In [ ]:
lagged_vars = ['lowest_price_per_mwh', 'highest_price_per_mwh', 
               'mean_price_per_mwh', 'var_price_per_mwh']
corr_gas_df = make_correlation_df(target_gas_price_df, 'target', lagged_vars, 24*7*4*6)

In [ ]:
corr_gas_df.head()

#### Plot of the Pearson correlation between consumption and lagged lowest gas prices vs. hourly lag

In [ ]:
mask = corr_gas_df['is_consumption'] == 1
g = sns.FacetGrid(corr_gas_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="corr_lowest_price_per_mwh")
g.add_legend()
g.fig.suptitle('Pearson correlation between consumption and lagged lowest gas prices')
g.fig.subplots_adjust(top=0.975)

#### Plot of the Pearson correlation between production and lagged lowest gas prices vs. hourly lag

In [ ]:
mask = corr_gas_df['is_consumption'] == 0
g = sns.FacetGrid(corr_gas_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="corr_lowest_price_per_mwh")
g.add_legend()
g.fig.suptitle('Pearson correlation between production and lagged lowest gas prices')
g.fig.subplots_adjust(top=0.975)

#### Plot of the Pearson correlation between consumption and lagged highest gas prices vs. hourly lag

In [ ]:
mask = corr_gas_df['is_consumption'] == 1
g = sns.FacetGrid(corr_gas_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="corr_highest_price_per_mwh")
g.add_legend()

#### Plot of the Pearson correlation between production and lagged highest gas prices vs. hourly lag

In [ ]:
mask = corr_gas_df['is_consumption'] == 0
g = sns.FacetGrid(corr_gas_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="corr_highest_price_per_mwh")
g.add_legend()

#### Plot of the Pearson correlation between consumption and lagged daily mean gas prices vs. hourly lag

In [ ]:
mask = corr_gas_df['is_consumption'] == 1
g = sns.FacetGrid(corr_gas_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="corr_mean_price_per_mwh")
g.add_legend()

#### Plot of the Pearson correlation between production and lagged daily mean gas prices vs. hourly lag

In [ ]:
mask = corr_gas_df['is_consumption'] == 0
g = sns.FacetGrid(corr_gas_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="corr_mean_price_per_mwh")
g.add_legend()

#### Plot of the Pearson correlation between consumption and lagged daily variation gas prices vs. hourly lag

In [ ]:
mask = corr_gas_df['is_consumption'] == 1
g = sns.FacetGrid(corr_gas_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="corr_var_price_per_mwh")
g.add_legend()

#### Plot of the Pearson correlation between production and lagged daily variation gas prices vs. hourly lag

In [ ]:
mask = corr_gas_df['is_consumption'] == 0
g = sns.FacetGrid(corr_gas_df[mask], col = 'prediction_unit_id', col_wrap = 2, 
                  sharex=False, sharey=False, aspect=2.2, height=3)
g.map_dataframe(sns.lineplot, x="hours_lag", y="corr_var_price_per_mwh")
g.add_legend()